<a href="https://colab.research.google.com/github/ananghw/data-indonesia/blob/master/WebScrapping_Simotandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prepare the library**

In [59]:
import requests
from bs4 import BeautifulSoup
%pip install tabula-py
%pip install JPype1
import pandas as pd
import tabula
from urllib.parse import urljoin
import os
import glob
from datetime import datetime
%pip install pandasql
from pandasql import sqldf

# **Build Dynamic URL**

In [42]:
base_url = 'https://simotandi.pertanian.go.id/datatabular.php'
page_url = 'https://simotandi.pertanian.go.id/datatabular_prov.php?no=57'

In [43]:
output_folder = 'hasil_scraping'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

kolom_header = [
    'No',
    'Provinsi/Kabupaten/Kota',
    'Bera',
    'Penggenangan',
    'Tanam (1-15 HST)',
    'Vegetatif 1 (16-30 HST)',
    'Vegetatif 2 (31-40 HST)',
    'Max. Vegetatif (41-54 HST)',
    'Generatif 1 (55-71 HST)',
    'Generatif 2 (72-110 HST)',
    'Panen',
    'Standing Crop',
    'Luas Baku Sawah (Ha)'
]

print(f"Mengambil halaman dari: {page_url}")
try:
    response = requests.get(page_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    pdf_links = soup.find_all('a', href=lambda href: href and '.pdf' in href)

    if not pdf_links:
        print("Tidak ada link PDF yang ditemukan di halaman ini.")
    else:
        total_pdfs = len(pdf_links)
        print(f"Ditemukan {total_pdfs} link PDF. Memulai proses ekstraksi...")

        for i, link_tag in enumerate(pdf_links):
            print(f"\n--- Memproses PDF {i+1}/{total_pdfs} ---")

            relative_link = link_tag['href']
            full_pdf_url = urljoin(base_url, relative_link)
            print(f"URL PDF: {full_pdf_url}")

            pdf_filename = os.path.basename(relative_link).replace('.pdf', '')
            output_excel_file = os.path.join(output_folder, f'tabel_dari_{pdf_filename}.xlsx')

            try:
                print("Mengekstrak tabel dari PDF...")
                tables = tabula.read_pdf(full_pdf_url, pages='all', lattice=True, multiple_tables=True, pandas_options={'header': None})

                if not tables:
                    print("Tidak ada tabel yang bisa diekstrak dari PDF ini.")
                else:
                    print(f"Berhasil! Ditemukan {len(tables)} tabel. Membersihkan dan menggabungkan...")

                    cleaned_tables_list = []

                    for j, df in enumerate(tables):
                        if df.empty:
                            continue

                        start_row_index = pd.to_numeric(df.iloc[:, 0], errors='coerce').first_valid_index()

                        if start_row_index is not None:
                            df_cleaned = df.loc[start_row_index:].reset_index(drop=True)

                            if len(df_cleaned.columns) == len(kolom_header):
                                df_cleaned.columns = kolom_header
                                # Masukkan tabel yang sudah bersih ke dalam list
                                cleaned_tables_list.append(df_cleaned)
                            else:
                                print(f"  Peringatan: Jumlah kolom di Tabel_{j+1} tidak cocok dengan header. Dilewati.")
                        else:
                            print(f"  Peringatan: Tidak ditemukan baris data numerik di Tabel_{j+1}. Dilewati.")

                    if cleaned_tables_list:
                        combined_df = pd.concat(cleaned_tables_list, ignore_index=True)

                        combined_df.to_excel(output_excel_file, index=False)
                        print(f"Semua tabel telah digabung dan disimpan ke: {output_excel_file}")
                    else:
                        print("Tidak ada tabel yang valid untuk digabungkan setelah proses pembersihan.")

            except Exception as e:
                print(f"GAGAL memproses PDF ini. Error: {e}")

except requests.exceptions.RequestException as e:
    print(f"Gagal mengakses URL halaman web: {e}")

print("\n\n--- Proses Selesai ---")

Mengambil halaman dari: https://simotandi.pertanian.go.id/datatabular_prov.php?no=57
Ditemukan 34 link PDF. Memulai proses ekstraksi...

--- Memproses PDF 1/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571104b571b7a13c5a05b7d5694c1e8af306.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 7 tabel. Membersihkan dan menggabungkan...
Semua tabel telah digabung dan disimpan ke: hasil_scraping/tabel_dari_571104b571b7a13c5a05b7d5694c1e8af306.xlsx

--- Memproses PDF 2/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571227552720a7e5efdc4adea1338f805220.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 10 tabel. Membersihkan dan menggabungkan...
Semua tabel telah digabung dan disimpan ke: hasil_scraping/tabel_dari_571227552720a7e5efdc4adea1338f805220.xlsx

--- Memproses PDF 3/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571356fd3896aaec780f1275d599f5fc416f.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 5 tabel. Membersihkan dan menggabungkan...
Semua tabel 

In [44]:
#CHECKING THE SAMPLE XLSX
excel_file_path = "/content/hasil_scraping/tabel_dari_571104b571b7a13c5a05b7d5694c1e8af306.xlsx"
df_sample = pd.read_excel(excel_file_path)
display(df_sample.head())

print(f"Number of rows: {df_sample.shape[0]}")
print(f"Number of columns: {df_sample.shape[1]}")

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha)
0,1,Aceh,74.988,16.344,18.000,15.951,12.391,18.769,17.068,13.864,26.228,96.043,214.729
1,2,Aceh Barat,3.643,1.192,932,672,329,746,785,477,1.348,3.941,10.192
2,3,Arongan Lambalek,375,109,28,23,22,138,215,50,217,476,1.188
3,4,Bubon,220,83,73,88,56,59,45,12,85,333,733
4,5,Johan Pahlawan,98,24,28,20,14,30,14,20,111,126,363


Number of rows: 313
Number of columns: 13


In [45]:
#COMBINE ALL THE DATA EXTRACTED


print("\nMenggabungkan seluruh file Excel menjadi satu file...")

# Cari semua file Excel di folder output
excel_files = glob.glob(os.path.join(output_folder, 'tabel_dari_*.xlsx'))

combined_all_df = []

for file in excel_files:
    try:
        df = pd.read_excel(file)
        df['Source File'] = os.path.basename(file)  # Tambahkan kolom sumber
        combined_all_df.append(df)
    except Exception as e:
        print(f"Gagal membaca {file}: {e}")

if combined_all_df:
    final_combined_df = pd.concat(combined_all_df, ignore_index=True)
    combined_output_file = os.path.join(output_folder, 'gabungan_semua_tabel.xlsx')
    final_combined_df.to_excel(combined_output_file, index=False)
    print(f"Berhasil menggabungkan semua file ke: {combined_output_file}")
else:
    print("Tidak ada file yang berhasil digabung.")

#CHECKING THE SAMPLE XLSX
excel_file_path = "/content/hasil_scraping/gabungan_semua_tabel.xlsx"
df_sample = pd.read_excel(excel_file_path)
display(df_sample.head())

print(f"Number of rows: {df_sample.shape[0]}")
print(f"Number of columns: {df_sample.shape[1]}")


Menggabungkan seluruh file Excel menjadi satu file...
Berhasil menggabungkan semua file ke: hasil_scraping/gabungan_semua_tabel.xlsx


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File
0,1,Maluku Utara,4.490,700,810,826,711,1.153,940,789,2.982,5.229,13.508,tabel_dari_5782329bff59db4b644faac7068039cd858...
1,2,Halmahera Barat,434,36,62,47,43,98,142,62,174,454,1.124,tabel_dari_5782329bff59db4b644faac7068039cd858...
2,3,Ibu,20,2,10,8,4,3,7,3,2,35,61,tabel_dari_5782329bff59db4b644faac7068039cd858...
3,4,Ibu Selatan,47,2,3,4,1,3,7,6,32,24,110,tabel_dari_5782329bff59db4b644faac7068039cd858...
4,5,Jailolo,122,11,23,19,22,57,62,33,65,216,417,tabel_dari_5782329bff59db4b644faac7068039cd858...


Number of rows: 7541
Number of columns: 14


In [58]:
#EXTRACT DATA PROVINSI
query = "SELECT * FROM df_sample WHERE No = '1'"
data_provinsi = sqldf(query)

result.head()
print(f"Number of rows: {data_provinsi.shape[0]}")
print(f"Number of columns: {data_provinsi.shape[1]}")

data_provinsi

Number of rows: 34
Number of columns: 14


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File
0,1,Maluku Utara,4.490,700,810,826,711,1.153,940,789,2.982,5.229,13.508,tabel_dari_5782329bff59db4b644faac7068039cd858...
1,1,Jawa Barat,297.601,50.775,61.931,90.249,97.112,108.186,69.095,42.460,109.888,469.033,937.352,tabel_dari_57327e852490f5963f684da96982b31b4a5...
2,1,Jambi,20.415,3.951,3.898,3.938,4.372,9.423,7.482,5.473,8.939,34.586,68.321,tabel_dari_57156af0039db656b9039528d1ebfbf6742...
3,1,Kepulauan Bangka Belitung,7.264,1.246,1.492,1.229,1.270,2.170,2.083,1.547,3.897,9.791,22.425,tabel_dari_5719b0c0ce8332390b45f8cc38096b94d95...
4,1,Kalimantan Timur,15328,2757,3123,2212,2398,3576,3436,2403,5581,17148,"41,316",tabel_dari_5764f724ac0091de42a82ea6713d6c0250e...
5,1,Sulawesi Tengah,37.728,7.695,8.564,9.295,7.564,11.400,9.523,7.047,17.486,53.393,117.157,tabel_dari_5772777369f7d5c69e4e1a6660960a7c6d2...
6,1,Kalimantan Selatan,"101,046","13,163","14,740","14,820","17,958","40,971","28,092","15,186","44,657","131,767","292,992",tabel_dari_5763567d57691a30da3afdbcddcea910c20...
7,1,Nusa Tenggara Barat,57.167,12.969,17.716,34.695,39.842,37.313,12.652,5.942,15.624,148.16,238.105,tabel_dari_575286cf254cca08b33e1620cfc64d5d49d...
8,1,Kalimantan Utara,4388,644,891,668,525,1197,1084,738,1620,5103,11904,tabel_dari_5765d6469192b3342ef2d07c57775a60293...
9,1,Sumatera Barat,65.947,18.033,13.668,9.309,8.559,17.623,14.100,10.008,36.329,73.267,194.760,tabel_dari_571356fd3896aaec780f1275d599f5fc416...


In [8]:
# prompt: save the gabungan_semua_tabel into google sheet

# Install gspread and google-auth
!pip install --upgrade gspread google-auth

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# Name of the sheet you want to save to
spreadsheet_name = 'data simotandi'
worksheet_name = 'Sheet1' # Or any other sheet name

try:
    # Try to open the spreadsheet by name
    spreadsheet = gc.open(spreadsheet_name)
except gspread.SpreadsheetNotFound:
    # If not found, create a new one
    print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new one...")
    spreadsheet = gc.create(spreadsheet_name)
    print(f"Spreadsheet '{spreadsheet_name}' created.")

# Select the worksheet. If it doesn't exist, create it (gspread v5+ handles this)
try:
    worksheet = spreadsheet.worksheet(worksheet_name)
except gspread.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found. Creating a new one...")
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed
    print(f"Worksheet '{worksheet_name}' created.")


# Assuming 'final_combined_df' is the DataFrame you want to save
# If it's not already defined, you need to ensure the preceding code
# successfully creates the 'final_combined_df' DataFrame.

if 'final_combined_df' in locals():
    print(f"Saving data to Google Sheet '{spreadsheet_name}' in worksheet '{worksheet_name}'...")

    # Clear existing data in the sheet
    worksheet.clear()

    # Convert DataFrame to a list of lists (including header)
    data_to_upload = [final_combined_df.columns.values.tolist()] + final_combined_df.values.tolist()

    # Upload the data
    worksheet.update(values=data_to_upload)

    print("Data successfully uploaded to Google Sheet.")
else:
    print("Error: 'final_combined_df' DataFrame was not created. Please ensure the preceding code ran successfully.")



Spreadsheet 'data simotandi' not found. Creating a new one...
Spreadsheet 'data simotandi' created.
Saving data to Google Sheet 'data simotandi' in worksheet 'Sheet1'...
Data successfully uploaded to Google Sheet.
